# Code pour scraper les datasets sur Thérèse Bonney et l'Exposition universelle de 1937 📷 🗃️

**Enjeux techniques** 👨‍💻
+ Automatiser la distinction entre les sources.
+ Automatiser le nommage des fichiers et leur recensement dans la colonne "FICHIER" du csv.
+ Obtenir des images de bonne qualité.
+ Pouvoir suivre les étapes du scrapping pour identifier les problèmes.


## /!\ Avertissement /!\ 🔺

Ce code utilise la bibliothèque **Selenium** 🌕 pour scrapper le site des bibliothèques spécialisées de la ville de Paris. Cette bibliothèque a pour but d'émuler le comportement d'un utilisateur sur une page web dynamique (en javascript). De ce fait elle a besoin d'un **path vers le driver de votre navigateur**. 

Il vous faut impérativement avoir installé **Chrome** (ou **Chromium** pour Linux) comme navigateur. Puis, il faut que vous téléchargiez le driver depuis le lien 🔗 ci-dessous, pour que le script de scrapping puisse fonctionner.

***Chrome driver :***

🔗 https://googlechromelabs.github.io/chrome-for-testing/

*Téléchargez le binary de "chromedriver" correspondant à votre OS, et exécutez l'installation.*


🖳 Sous linux téléchargez **Chromium** via le centre d'application ou en suivant ce lien :

🔗 https://code.launchpad.net/~chromium-team/chromium-browser/+git/snap-from-source

## Environnement 🌐

### Import des librairies 📖

In [13]:
import os
import re
import time
import shutil
import zipfile
import tempfile
import requests
import pandas as pd
from random import randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

## Fonctions 🤖

### Fonction principale de téléchargement d'images 💾

In [14]:
def download_image(input_csv, start_index=0): #On a besoin de start_index pour pouvoir remplacer cette valeur en cas d'interruption du scrapping
    """
    Cette fonction lit un fichier CSV contenant des URLs d'images et leurs notices associées,
    puis télécharge chaque image selon sa source (Gallica, Bibliothèques, etc.) en appelant
    la fonction de téléchargement appropriée.

    Args:
        input_csv (str): Chemin vers le fichier CSV contenant les URLs à traiter.
        start_index (int) : Index de départ utilisé pour lancer le téléchargement.
        

    Returns:
        bool: False uniquement en cas d'erreur lors de la lecture du fichier CSV.
              Sinon, la fonction ne retourne rien (None).
    """
    # Définition dynamique du chemin vers le folder d'output 

    output_folder = os.path.join(os.path.dirname(input_csv), 'sortie') # Le dossier de sortie se trouvera au même endroit que le csv qui contient les URLs.
    os.makedirs(output_folder, exist_ok=True) #S'il n'existe pas, il est créé.

    try:
        df = pd.read_csv(input_csv, sep=",", encoding="utf-8") # Ouverture du csv avec pandas comme DataFrame ⚠️ Utilisez le bon séparateur ⚠️
        print("Fichier CSV chargé avec succès.")
        if "FICHIER" not in df.columns: # Création de la colonne fichier qui viendra contenir les filepaths des images téléchargées.
            df["FICHIER"] = ""
        for index, row in df.iterrows(): # On lui dit d'itérer sur chaque ligne.
            if index < start_index: # L'index démarre par défaut à zéro, mais si la variable start_index est supérieure à 0 on ignore les lignes entre 0 et start_index.
                continue 
            try: # On vérifie quelle fonction on va appeler pour le téléchargement en fonction du site d'origine de l'image.
                url = row['URL']
                output_path = os.path.join(output_folder, f"image_{index:02d}.jpg") # On nomme les fichiers d'images en suivant l'index du csv.
                if url.startswith("https://gallica"): # On détecte l'URL de Gallica.
                    success = download_gallica(url, index, output_folder) # Et on appelle la fonction correspondante.
                    if success:
                        df.at[index, "FICHIER"] = output_path # Si le téléchargement réussi on ajoute à la ligne correspondante le filepath de l'image dans la colonne "FICHIER"
                        df.to_csv(input_csv, index=False, encoding="utf-8") # On sauvegarde les modifications.
                elif url.startswith('https://bibliotheques'): # Autrement on détecte que l'image provient du site des bibliothèques spécialisées de Paris. 
                    success = download_bib(url, index, output_folder, output_path) # On appelle la fonction idoine.
                    if success:
                        df.at[index, "FICHIER"] = output_path # Pareil on ajoute le filepath à la colonne "FICHIER"
                        df.to_csv(input_csv, index=False, encoding="utf-8") # On sauverharde les modifications.
                else:
                    print(f"Source non reconnue pour l'URL au n° d'index {index}") # Si une URL d'un autre site est présente dans notre CSV, ce message d'erreur nous permettra de savoir à quelle index du csv se trouve l'URL.
            except Exception as e:
                print(f"Erreur ligne {index}: {e}")
                continue
    except Exception as e:
        print(f"Erreur lors de la lecture du CSV : {e}") # Si le CSV ne s'est pas lu correctement, par exemple parce si le séparateur n'est pas le bon, ou l'encodage différent.   
        return False


### Fonction spécifique pour Gallica 🐓

In [15]:
def download_gallica(url, index, output_folder):
    """
    Télécharge une image depuis Gallica
    
    Args:
        url (str): URL originale de Gallica
        index (int): Numéro de ligne dans le CSV
        output_folder (str): Dossier de destination
        
    Returns:
        bool: True si succès, False si échec
    """
    try:
        # 1. Transformer l'URL
        ark = url.split('ark:') # On isole dans la colonne URL la partie de l'url qui contient l'identifiant ark de l'image.
        if len(ark) <= 1: # On vérifie qu'on obtient pas une résultat vide.
            print(f"URL invalide à la ligne {index}") # Le cas échéant on imprime une erreur avec le numéro d'index pour voir d'où vient le problème.
            return False
        image_url = f"https://gallica.bnf.fr/iiif/ark:{ark[1]}/f1/full/max/0/default.jpg" # Sinon on forme l'URL IIIF de l'image pour l'ouvrir directement en plein écran et en bonne qualité pour le téléchargement.

        # 2. Télécharger
        timeout = randint(4,9) # Afin d'éviter un code 429 (Too many requests) de la part de Gallica on randomise le timeout entre les téléchargements.
        response = requests.get(image_url, stream=True, timeout=timeout) # On envoie la requête pour accéder à la ressource auprès du site.
        if response.status_code != 200: # On s'assure qu'on obtient bien un code 200, signifiant que tout est ok.
            print(f'Problème url {index}: status code {response.status_code}') # Sinon on print l'index de la ligne et le code d'erreur obtenu.
            return False

    
        # 3. Sauvegarder
        output_path = os.path.join(output_folder, f"image_{index:02d}.jpg") # On créer le nommage régulier des fichiers dans le chemin que nous avons défini dynamiquement dans la fonction générale.
        with open(output_path, 'wb') as file: # On ouvre en mode écriture binaire le fichier.
            for chunk in response.iter_content(1024): # On charge par chunks (morceaux) afin d'éviter de devoir charger en mémoire l'entièreté de l'image avant de la télécharger, ce qui peut être lourd et long avec des images de bonne qualité.
                file.write(chunk) # On télécharge.
        
        print(f"Image de Gallica téléchargée avec succès n° d'index {index} ") # Tout s'est bien passé, et on imprime le numéro d'index de la ligne pour suivre où nous en sommes.
        time.sleep(timeout) # On laisse un temps de pause entre deux téléchargements, toujours pour éviter un code 429.
        return True

    except Exception as e:
        print(f"Erreur pour l'URL Gallica n° d'index {index}: {e}") # Quelque chose ne s'est pas bien passé on print le numéro d'index et l'erreur rencontrée.
        return False          


### Fonctions spécifiques pour les bibliothèques spécialisées de Paris 🗼

#### Fonction pour gérer les cookies 🍪

In [16]:
def dismiss_cookies(driver):
    """
    Cette fonction tente de fermer ou d'ignorer une fenêtre de cookies en cliquant
    sur le corps de la page, ce qui peut suffire à la faire disparaître sur certains sites.

    Args:
        driver (selenium.webdriver): Instance active du navigateur pilotée par Selenium.

    Returns:
        None
    """
    wait = WebDriverWait(driver, 5) # On attends 5 secondes que la page se charge.
    body = wait.until(EC.presence_of_element_located((By.TAG_NAME, "body"))) # Dès que l'élément "body" de la page HTML apparaît on arrête d'attendre.
    ActionChains(driver).move_to_element(body).click().perform() # On clique dès lors n'importe où sur le corps de la page pour faire disparaître les cookies.

#### Fonction pour dézipper, déplacer et renommer les fichiers 🤐

In [17]:
def process_zip_file(zip_path, index, output_folder, timeout=30):
    """
    Traite le fichier ZIP téléchargé
    
    Args:
        zip_path (str): Chemin du fichier ZIP téléchargé
        index (int): Index pour le nommage
        output_folder (str): Dossier de destination final
        timeout (int): Temps maximum d'attente en secondes
    
    Returns:
        bool: True si succès, False si échec
    """
    try:
        start_time = time.time() # Sert à commencer à décompter le temps, on utilise un "timeout" plus loin, la fonction a donc besoin de savoir combien de temps s'est écoulé depuis qu'elle a démarrée.
        

        if not os.path.exists(zip_path): # Vérification que le ZIP existe.
            print(f"ZIP non trouvé: {zip_path}") # Sinon on dit où est-ce qu'on a pas trouvé le ZIP.
            return False
            
        with tempfile.TemporaryDirectory() as temp_dir: # On crée un dossier temporaire pour dézipper le dossier.
            with zipfile.ZipFile(zip_path, 'r') as zip_ref: # On accède en lecture au dossier zip.
                zip_ref.extractall(temp_dir) # On extrait le dossier. 
            
            image_found = False
            for root, _, files in os.walk(temp_dir): # On parcours le dossier extrait pour y trouver l'image
                if time.time() - start_time > timeout: # On met une limite de temps à ce traitement.
                    print(f"Timeout lors du traitement du ZIP n° d'index {index}") # Si aucun fichier n'est trouvé après le temps imparti on imprime un message d'erreur avec l'index.
                    return False                    
                for file in files:
                    if file.lower().endswith(('.jpg', '.jpeg', '.png')): # On cherche les fichiers qui finissent par ...
                        # 4. Déplacer et renommer l'image
                        src = os.path.join(root, file) # On prend le fichier à déplacer dans le dossier temporaire, la variable src étant le chemin complet de ce dossier temporaire.
                        dst = os.path.join(output_folder, f"image_{index:02d}{os.path.splitext(file)[1]}") # Ici c'est le chemin de destination de l'image et la norme de nommage de l'image au moment du déplacement.
                        shutil.move(src, dst) # On bouge et renomme le fichier depuis le dossier temporaire jusqu'au dossier de sortie final.

# 💬 Ici on ne supprime pas directement le dossier zip des Téléchargements, on le fait plus tard, dans la fonction downloab_bib.
                  
                        
                        if os.path.exists(dst): # On vérifie que l'image a bien été déplacée et existe
                            image_found = True
                            print(f"Image {index} extraite et déplacée avec succès ✅") # Si c'est le cas on imprime un message de réussite.
                            break # On casse la boucle "for file in files:" quand une image a été trouvée, déplacée et renommée.
                        
                        # Le code reprend ici à la boucle supérieure "for root, _, files in os.walk(temp_dir):"
                
            if not image_found:
                print(f"Aucune image trouvée dans le ZIP n° d'index {index}")
                return False # Ici on met fin à la boucle citée précédemment, soit parce que ça a échoué.
                
            return True # Soit parce que ça a réussi.
        
    except Exception as e:
        print(f"Erreur traitement ZIP n° d'index {index}: {e}") # Gestion d'erreur, en cas de problème à un quelconque moment de la fonction, déclenche ce message d'erreur avec l'index et le message produit.
        return False

#### Fonction pour détecter le webdriver installé sur la machine 🚀

In [18]:
def find_webdriver(): # Afin d'assurer la portabilité du code nous devons prendre en compte les différentes types d'OS.

# On met en clair dans le code les chemins classiques, attendus, que la fonction download_bib pourra interroger afin de lancer Chromedriver.

# 👁️‍🗨️⚠️ Si vous installez Chromedriver à un endroit personnalisé, vous devrez mentionner son path ici pour que le scrapping puisse fonctionner.
    chrome_paths = [
        "/usr/local/bin/chromedriver",  # Linux/Mac
        "/usr/bin/chromedriver",        # Linux
        "C:\\Program Files\\Google\\Chrome\\Application\\chromedriver.exe",  # Windows
        "C:\\chromedriver.exe"          # Windows
    ]
    for path in chrome_paths:
        if os.path.exists(path): # On vérifie que Chromedriver existe bel et bien à l'un de ces chemins.
            return path
    raise FileNotFoundError("Aucun chromedriver trouvé. Installez chromedriver. Vérifiez qu'il se situe bien aux endroits attendus dans la variable 'chrome_paths'") # Le cas échéant nous invitons l'utilisateur à réinstaller Chromedriver, en précisant cette fois son path ci-dessus.

#### Fonction pour gérer les pages contenant plusieurs images 🖼️

In [19]:
def handle_multi_img_case(driver, index):
    """
    Si plusieurs images sont présentes sur la page, clique sur la première,
    gère le basculement d'onglet, ferme l'onglet original, puis retourne True si succès, False sinon.
    
    Args:
        driver (selenium.webdriver): Instance active du navigateur Selenium.
        index (int): Index de la ligne en cours (pour le suivi/log).

    Returns:
        bool: True si le switch et la fermeture de l'onglet original ont réussi, False sinon.
    """
    try:
        container_xpath = "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/main/div/div/div/div/div/div[2]/div/div/div[1]/div/div/div[1]/div/div[2]"
        container = driver.find_element(By.XPATH, container_xpath) # Si la page a plusieurs images c'est toujours dans cette div
        img_elements = container.find_elements(By.TAG_NAME, "img") # Il y aura la présence de plusieurs img=class dans ce cas
        if len(img_elements) > 1: # Donc si img_elements est supérieur à 1
            img_elements[0].click() # On clique sur le premier
            print(f"Plusieurs images détectées, première image cliquée au n° d'index {index}")
            original_window = driver.current_window_handle # On définit une variable original_window pour gérer la fermeture des fenêtres plus tard, on lui dit que c'est celle qu'on manipule maintenant
            time.sleep(1) # On attend.
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1) # Ici on attends jusqu'a ce qu'il y ait plus d'un onglet.
            time.sleep(1)
            # Trouver la nouvelle fenêtre
            new_window = None # Pour l'instant on définit que new_window est vide pour le peupler après.
            for window_handle in driver.window_handles:
                if window_handle != original_window: # Ici on définit que si la page sur laquelle on est est différente de celle de case alors c'ets la nouvelle page
                    new_window = window_handle
                    break
            if new_window is None: # Ici c'est si ça n'a pas marché on lui retourne un message d'erreur.
                print(f"Impossible de trouver la nouvelle fenêtre au n° d'index {index}")
                return False
            driver.switch_to.window(new_window) #Ici on switch sur ce qu'on a définit comme étant le nouvel onglet.
            time.sleep(1)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body")) # Ici on attends jusqu'à ce qu'on ait le body.
            )
            time.sleep(1)
            # Fermer l'onglet original
            driver.switch_to.window(original_window) # On bascule sur l'onglet de base.
            time.sleep(1)
            driver.close() # On le ferme.
            time.sleep(1)
            # Revenir sur le nouvel onglet
            driver.switch_to.window(new_window) # Enfin on reviens sur le nouvel onglet.
            time.sleep(1)
            print(f"Basculé sur la nouvelle fenêtre et fermé l'onglet original pour l'image au n° d'index {index}")
            return True
        else:
            return False
    except Exception as e:
        print("Pas de gestion multi-image pour cette URL")
        return False

#### Fonction de crawling et de téléchargement 🦀

In [20]:
def download_bib(url, index, output_folder, output_path): # Fonction qui automatise la navigation sur les pages web du site des bibliothèque de Paris.
    """
    Fonction pour interagir avec les pages des bibliothèques spécialisées de Paris

        Args:
        url (str): URL originale de Gallica
        index (int): Numéro de ligne dans le CSV
        output_folder (str): Dossier de destination
        output_path (str): nom du filepath des images téléchargées
    Returns:
        bool: True si succès, False si échec
    """
    try:
        driver_path = find_webdriver() # On cherche l'instance chromedriver en fonction du path qui a été trouvé dans la fonction précédente.
        user_data_dir = tempfile.mkdtemp() # On créer un instance temporaire avec des informations de profil temporaires.

        options = webdriver.ChromeOptions() # On liste les options de lancement de Chrome via Selenium.
        options.add_argument("--no-sandbox") #Désactive le mode "sandbox" de Chrome, qui "conteneurise" les onglets et les pages pour éviter les conflits de lancement notamment en serveur docker ou sur Linux.
        options.add_argument(f"--user-data-dir={user_data_dir}") # Spécifie un dossier temporaire pour le profil utilisateur Chrome. Cela permet d’isoler la session de navigation de Selenium du profil Chrome habituel.
        options.add_argument("--headless=new") # N'affiche pas l'interface graphique, possibilité de commenter la ligne pour voir le scrapping se faire.
        options.add_argument("--disable-dev-shm-usage") # Empêche Chrome d’utiliser /dev/shm (mémoire partagée). Sur certains systèmes (ex : Docker), la mémoire partagée est limitée et cela évite des plantages.
        options.add_argument("--disable-gpu") #Désactive l’accélération matérielle GPU. Recommandé en mode headless ou sur des serveurs sans GPU.
        options.add_argument("--remote-debugging-port=9222") # Ouvre le port 9222 pour le débogage à distance de Chrome. Nécessaire pour certaines fonctionnalités avancées ou pour le support du mode headless.
        
# ⚠️ Par défaut on indique le chemin vers le dossier des Téléchargements pour que Chrome sache où mettre les dossiers qu'on lui demande de télécharger.
# ⚠️ Possibilité de changer le chemin pour éviter les conflits si d'autres dossiers .zip se trouvent dans vos Téléchargements. Sinon ce n'est qu'un dossier intérmédiaire, pas besoin d'un dossier dédié, les .zip sont supprimés après chaque téléchargement.      
        
        # Préférences pour le téléchargement automatique des ZIP
        prefs = {
            "download.default_directory": os.path.expanduser("~/Téléchargements"), # On indique au navigateur la destination que doivent prendre les fichiers téléchargés.
            "download.prompt_for_download": False, # Ne demande pas à l'utilisateur, via un pop-up, où télécharger un fichier.
            "download.directory_upgrade": True, # Permet d'utiliser le nouveau dossier de téléchargement même si un téléchargement est déjà en cours.
            "safebrowsing.enabled": True # Active la protection contre les fichiers malveillants
        }
        options.add_experimental_option("prefs", prefs) # On applique le contenu de notre variable prefs à la méthode qui régit les options de lancement Chrome.

        service = Service(driver_path) # On dit que 'service' est de la classe Service, qui lance ou coupe le navigateur et on lui donne comme argument le chemin vers le driver pour qu'il puisse s'exécuter.
        driver = webdriver.Chrome(service=service, options=options) # On lance le driver via Selenium avec nos options précédemment définies.

        try:
            # 1. Charger la page
            print(f"Chargement de la page {index}...")
            driver.get(url) # On se connecte à l'URL.
            time.sleep(5) # On attends 5 secondes.
            # Vérifier que la page est bien chargée
            if "Erreur" in driver.title or "404" in driver.title:
                print(f"Problème de chargement de la page au n° d'index {index} (titre: {driver.title})")# On fait de la gestion d'erreur où on affiche le titre de la page et l'index.
            else:
                print(f"Page chargée avec succès au n° d'index {index} (titre: {driver.title})") # On manifeste la réussite du chargement avec l'index et le titre correspondant.

            # 2. Gérer les cookies
            try:
                dismiss_cookies(driver) # On fait appel à notre fonction pour dissiper la fenêtre des cookies qui apparaît.
            except Exception as e:
                print(f"Erreur cookies: {e}, on continue...")
                    
            # 3. Afficher l'image en grand dans une autre fenêtre
            if handle_multi_img_case(driver, index):
            # Si la fonction a géré le cas multi-img, passer directement à l'étape 5 (bouton téléchargement)
                pass  # Le code continue à l'étape 5
            try:
                image = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//*[@id='fullscreen']"))
                )
                image.click()
                print(f"Bouton fullscreen cliqué n° d'index {index}")
            except Exception:
                try:
                    image = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/main/div/div/div/div/div/div[1]/div[2]/div/div[1]/img"))
                    )
                    print(f"Image trouvée par XPATH principal (classique) n° d'index {index}")
                except Exception:
                    try:
                        image = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/main/div/div/div/div/div/div[1]/div[1]/div/div[1]/img"))
                        )
                        print(f"Image trouvée par XPATH alternatif (classique) n° d'index {index}")
                    except Exception:
                        try:
                            image = WebDriverWait(driver, 5).until(
                                EC.element_to_be_clickable((By.XPATH, "//img[@role='presentation']"))
                            )
                            print(f"Image trouvée par rôle 'presentation' (classique) au n° d'index {index}")
                        except Exception:
                            print(f"Impossible de trouver l'image au n° d'index {index} avec toutes les méthodes")
                            raise Exception("Image non trouvée")
                if image.tag_name == "img":
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", image)
                    time.sleep(0.5)  # Facultatif mais souvent utile
                    try:
                        image.click()
                    except Exception as e:
                        print(f"Click classique échoué, tentative JS click... {e}")
                        driver.execute_script("arguments[0].click();", image)
                    print(f"Image cliquée au n° d'index {index}")
                else:
                    print(f"Élément trouvé mais ce n'est pas une image ligne {index}")
                    raise Exception("Pas un élément image")
            except Exception as e:
                print(f"Impossible de trouver l'image au n° d'index {index}: {e}")
                raise

            # 4. Basculer sur la nouvelle fenêtre
            try:
                WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1) # Ici il faut qu'on bascule sur une fenêtre pop-up pour la suite du scrapping.
                # On attends donc 10 secondes le chargement puis on passe par une fonction lambda qui simplement nous permet de basculer à l'autre fenêtre s'il y a plus d'une fenêtre en même temps (donc si la deuxième est apparue)
                # La session Chrome est stoppée après chaque téléchargement donc il n'y aura jamais plus de deux onglets sur la même fenêtre c'est pour cela qu'on peut passer par cette fonction lambda.
                original_window = driver.current_window_handle # On définit cette variable pour dire que la page sur laquelle on était, était la page originale, celle qu'on va donc quitter.
                for window_handle in driver.window_handles:
                    if window_handle != original_window: # Si donc il y a une seconde fenêtre différente de celle originale.
                        driver.switch_to.window(window_handle) # On utilise la méthode qui permet de passer d'une fenêtre à l'autre.
                        break
                print(f"Basculé sur la nouvelle fenêtre pour l'image au n° d'index {index}")
                    
            except Exception as e:
                print(f"Impossible de basculer sur la nouvelle fenêtre au n° d'index {index}: {e}")
                raise

            # 5. Appuyer sur le bouton "Téléchargement"
            try:
                dl = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//*[@id='hires']"))
                ) # Même logique que pour afficher l'image en grand, on attend que l'élément du bouton de téléchargement soit chargé.
                if dl.tag_name == "div": # On s'assure qu'il est de la bonne nature. 
                    dl.click() # On clique dessus.       
                    print(f"Bouton téléchargement trouvé et cliqué au n° d'index {index}")
                else:
                    print(f"Élément trouvé mais ce n'est pas un bouton de téléchargement {index}")
                    raise Exception("Pas un bouton de téléchargement") # Ce qu'on a trouvé n'était pas de la bonne nature.
                
            except Exception as e:
                print(f"Impossible de trouver le bouton de téléchargement au n° d'index {index}: {e}")
                raise

            # 6. Cocher la checkbox
            try:
                checkbox = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//*[@id='checkConditions']"))
                ) # Même logique qu'avant, on doit attendre qu'un nouvel élément dynamique apparaisse, cette fois-ci c'est une case à cocher.
                if checkbox.get_attribute('type') == "checkbox": # On vérifie sa nature.
                    checkbox.click() # Si la nature est la bonne, on clique.       
                    print(f"Checkbox des conditions trouvée et cochée au n° d'index {index}")
                else:
                    print(f"Élément trouvé mais ce n'est pas une checkbox {index}")
                    raise Exception("Pas une checkbox")
                
            except Exception as e:
                print(f"Impossible de trouver la checkbox au n° d'index {index}: {e}")
                raise
            
            # 7. Appuyer sur le bouton download
            try:
                dl_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/div[4]/div[3]/div/button"))
                ) # Même logique, on attends que le bouton de download final apparaisse et on clique dessus.
                if dl_button.tag_name == "button": # On vérifie sa nature.
                    dl_button.click() # On clique.       
                    print(f"Bouton de téléchargement final trouvé et cliqué au n° d'index {index}")
                else:
                    print(f"Élément trouvé mais ce n'est pas le bouton pour télécharger {index}")
                    raise Exception("Pas un bouton de téléchargement")
                
            except Exception as e:
                print(f"Impossible de trouver le button pour télécharger l'image au n° d'index {index}: {e}")
                raise

            # 8. Attendre le téléchargement et traiter le ZIP
            try:
                # Attendre pour le téléchargement
                time.sleep(15)  
                
                # Vérifier que le dossier Téléchargements existe
                downloads_dir = os.path.expanduser("~/Téléchargements")
                if not os.path.exists(downloads_dir):
                    raise Exception(f"Dossier Téléchargements non trouvé: {downloads_dir}")
                    
                # Vérifier qu'il y a des fichiers dans le dossier
                files = os.listdir(downloads_dir)
                if not files:
                    raise Exception("Dossier Téléchargements vide")
                    
                # Trouver le dernier fichier ZIP
                zip_files = [f for f in files if f.endswith('.zip')]
                if not zip_files:
                    raise Exception("Aucun fichier ZIP trouvé dans Téléchargements")
                    
                latest_zip = max([os.path.join(downloads_dir, f) for f in zip_files], 
                                key=os.path.getctime) # Cette ligne crée une liste des chemins complets de tous les fichiers ZIP présents dans le dossier de téléchargement,
# puis sélectionne celui qui a la date de création la plus récente (le dernier ZIP téléchargé).
# Cela permet de cibler automatiquement le fichier ZIP qui vient d'être téléchargé pour le traiter ensuite.
                
                print(f"Fichier ZIP trouvé: {latest_zip}")
                
                # Traiter le ZIP
                if not process_zip_file(latest_zip, index, output_folder):
                    raise Exception("Échec du traitement du ZIP")
                    
                # Suppression du ZIP traité
                os.remove(latest_zip)
                print(f"ZIP supprimé avec succès")
                            
            except Exception as e:
                print(f"Erreur traitement fichier téléchargé au n° d'index {index}: {e}")
                raise

            return True # On retourne que tout s'est bien passé.
            
        finally:
            driver.quit() # On ferme le chromedriver.
            
    except Exception as e:
        print(f"Erreur configuration Selenium au n° d'index {index}: {e} ❌")
        return False

### Fonction pour reprendre une session de scrapping interrompue 🤕

In [21]:
def resume_scrapping(output_folder):
    """
    Cette fonction sert à reprendre un téléchargement en cours de route s'il s'est arrêté subitement.
    Elle retourne l'index de la dernière image téléchargée dans le dossier de sortie.
    Args:
        output_folder (str): Dossier de destination des images téléchargées.
    Returns:
        int: L'index le plus élevé trouvé dans les noms de fichiers, ou -1 si aucun fichier n'est trouvé.
    """
    
    files = os.listdir(output_folder) # On liste tous les fichiers présents dans le dossier de sortie.
   
    max_index = -1  # Initialise la variable max_index à -1, afin de s'assurer qu'on a pas encore d'image dans le dossier.

    for name in files:     # Parcourt chaque nom de fichier dans le dossier.
       
        match = re.match(r'image_(\d+)\.\w+', name)  # Cherche un nom de fichier qui correspond au format "image_<nombre>.<extension>".
        if match: # Si le nom correspond, extrait l'index numérique du nom de fichier.
            idx = int(match.group(1))   # Met à jour max_index si l'index trouvé est supérieur à l'actuel.
            if idx > max_index:
                max_index = idx
   
    return max_index  # Retourne l'index le plus élevé trouvé (ou -1 si aucun fichier ne correspond).

## Processing ⚙️

In [ ]:
input_csv = "" # Chemin absolu vers le csv qui contient les adresses URL des images.

### Lancement du scrapping pour la première fois 🎂

**S'il a déjà été lancé, qu'il s'est stoppé subitement ou que nous l'avons arrêté sciemment, pour le reprendre là où il s'est arrêté, décommentez la cellule suivante 🩹 et reprenez le téléchargement en l'exécutant.**

⚠️🌪️ Ici si vous relancez un scrapping déjà entamé avec la cellule ci-dessous, il recommencera du début et écrasera les photos déjà présentes. 🌪️⚠️ 

In [ ]:
# Lancer le scrapping pour la première fois
download_image(input_csv)

### Relancer le scrapping en cas d'interruption 🔂

🩹
**Décommentez ci-dessous et exécutez la cellule pour reprendre un scrapping interrompu**

In [ ]:
# # Reprendre un scrapping interrompu
# output_folder = os.path.join(os.path.dirname(input_csv), 'sortie') # On met le chemin de sortie, c'est-à-dire dans le même dossier que le csv.
# last_index = resume_scrapping(output_folder) # Via la fonction resume_scrapping, on récupère le plus haut index trouvé dans le dossier.
# download_image(input_csv, start_index=last_index+1) # On reprend le téléchargement en attribuant la valeur last_index à la variable start_index, de notre fonction principale de téléchargement, qui est originellement 0 par défaut.